In [4]:
import pandas as pd

In [5]:
import matplotlib.pyplot as plt

In [6]:
import seaborn as sns

In [7]:
bom_movie_gross = pd.read_csv("../data/bom.movie_gross.csv.gz")

In [8]:
imdb_name_basics = pd.read_csv("../data/imdb.name.basics.csv.gz")

In [9]:
imdb_title_akas = pd.read_csv("../data/imdb.title.akas.csv.gz")

In [10]:
imdb_title_basics = pd.read_csv("../data/imdb.title.basics.csv.gz")

In [11]:
imdb_title_crew = pd.read_csv("../data/imdb.title.crew.csv.gz")

In [12]:
imdb_title_principals = pd.read_csv("../data/imdb.title.principals.csv.gz")

In [13]:
imdb_title_ratings = pd.read_csv("../data/imdb.title.ratings.csv.gz")

In [14]:
rt_movie_info = pd.read_csv("../data/rt.movie_info.tsv.gz", sep='\t')

In [39]:
rt_reviews = pd.read_csv(data_directory+'rt.reviews.tsv.gz', delimiter = '\t', encoding = 'cp437')

In [16]:
tmdb_movies = pd.read_csv("../data/tmdb.movies.csv.gz")

In [17]:
tn_movie_budgets = pd.read_csv("../data/tn.movie_budgets.csv.gz")

In [18]:
%matplotlib inline

In [32]:
import pandas as pd
data_directory = '../Data/'  
# df_bom = pd.read_csv(data_directory+'bom.movie_gross.csv.gz') #Source: Box Office Mojo
# df_rt_movies = pd.read_csv(data_directory+'rt.movie_info.tsv.gz', delimiter = '\t') #Source: Rotten tomatoes
# df_rt_reviews = pd.read_csv(data_directory+'rt.reviews.tsv.gz', delimiter = '\t', encoding = 'cp437') #Source: Rotten tomatoes 
df_imdb_basics = pd.read_csv(data_directory+'imdb.title.basics.csv.gz') # Source: IMDB
df_imdb_ratings = pd.read_csv(data_directory+'imdb.title.ratings.csv.gz') # Source: IMDB
df_mdb_ratings = pd.read_csv(data_directory+'tmdb.movies.csv.gz') # Source: MovieDB
df_mdb_revenues = pd.read_csv(data_directory+'tn.movie_budgets.csv.gz') # Source: MovieDB

# Creating Lists and Graphs and Things

Instead of using the arbitrary 100000 we will use 25%... 

So I want to first sort by "averagerating", because if we're going to claim to know what type of movie Microsoft Film Studios should make like some kind of mystic, we may as tell them we aim to be compared to the best of the best.  But they also need to have a certain amount of weight, or proof which comes for better or worse throgh "numvotes", because strength in numbers. 

# Plot average rating over the y-axis, movies with greater "numvotes" have a more opaque color whereas the movies with less are transparent. X-Axis displays names? vvvv

In [33]:
# The function cleans the two Movie DB dataframes
def df_clean(data_frame, release_date_column, movie_title_column):
    data_frame.drop_duplicates(keep = 'first', inplace = True)
    if data_frame[release_date_column].dtypes == 'int64':
        data_frame[release_date_column] = pd.to_datetime(data_frame[release_date_column], format = '%Y').dt.to_period("Y") # Assume that movies released in the same month of the same year are the same.
    else:
        data_frame[release_date_column] = pd.to_datetime(data_frame[release_date_column]).dt.to_period("Y") # Assume that movies released in the same month of the same year are the same.
    data_frame[movie_title_column] = [" ".join(title.upper().split()) for title in data_frame[movie_title_column]] # Remove whitespaces
    data_frame[movie_title_column] = [title.replace('Â', "'") for title in data_frame[movie_title_column]] 
    data_frame[movie_title_column] = [title.replace('#', '') for title in data_frame[movie_title_column]] 
    data_frame[movie_title_column] = [title.replace('"', '') for title in data_frame[movie_title_column]]               
    data_frame.drop_duplicates([movie_title_column, release_date_column], keep = False, inplace = True)
    data_frame.sort_values(by = [release_date_column, movie_title_column], ascending = [False, True], inplace = True)
    data_frame.rename(columns={release_date_column: 'release_date', movie_title_column: 'title'}, inplace = True)
    return data_frame

In [34]:
# Merge Movie DB dataframes
df_mdb_ratings.drop('Unnamed: 0', axis = 1, inplace = True) # This is an extra index column
df_mdb_ratings = df_clean(df_mdb_ratings, 'release_date', 'title')
df_mdb_revenues = df_clean(df_mdb_revenues, 'release_date', 'movie')
df_mdb = pd.merge(df_mdb_ratings, df_mdb_revenues, how = 'outer', \
              left_on = ['title', 'release_date'], \
              right_on = ['title', 'release_date'], \
              indicator = True, validate="one_to_one", \
              sort = True)


In [35]:
# Merge IMDB dataframes
df_imdb_basics = df_clean(df_imdb_basics, 'start_year', 'primary_title')
df_imdb = pd.merge(df_imdb_basics, df_imdb_ratings, how = 'outer', \
              on = 'tconst', \
              indicator = True, validate="one_to_one")

In [36]:
# Merge Movie DB and IMDB dataframes
df = pd.merge(df_imdb.drop(columns = '_merge'), df_mdb.drop(columns = '_merge'), how = 'outer', \
              left_on = ['title', 'release_date'], \
              right_on = ['title', 'release_date'], \
              indicator = True, validate="many_to_one", \
              sort = True)
df_merged = df.loc[df._merge == 'both']

In [47]:
df_merged = df.loc[df._merge == 'both']


# Took out any movies that didn't have rating votes in either IMDB or MoviesDB

In [55]:
(df_merged['vote_average'].isna() & df_merged['numvotes'].isna()).sum()

17

In [51]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12125 entries, 3 to 159008
Data columns (total 20 columns):
tconst               12125 non-null object
title                12125 non-null object
original_title_x     12124 non-null object
release_date         12125 non-null period[A-DEC]
runtime_minutes      11882 non-null float64
genres               12081 non-null object
averagerating        11867 non-null float64
numvotes             11867 non-null float64
genre_ids            11992 non-null object
id_x                 11992 non-null float64
original_language    11992 non-null object
original_title_y     11992 non-null object
popularity           11992 non-null float64
vote_average         11992 non-null float64
vote_count           11992 non-null float64
id_y                 1504 non-null float64
production_budget    1504 non-null object
domestic_gross       1504 non-null object
worldwide_gross      1504 non-null object
_merge               12125 non-null category
dtypes: category(

In [52]:
258/12125

0.021278350515463916